In [1]:
import sys
sys.path.append(r"C:\Users\hjia9\Documents\GitHub\data-analysis")
sys.path.append(r"C:\Users\hjia9\Documents\GitHub\data-analysis\ucla-lapd")

import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d, gaussian_filter
from scipy.signal import find_peaks, savgol_filter

import read_hdf5 as rh
from Jan2024_Isat import init_read, get_Isat_ratio, setup_2subplots, get_IV, find_IV_tndx, find_Te, get_Te_all
from bapsflib import lapd
from lp_analysis import EEDF, derivative, find_Vp


%matplotlib qt
plt.rcParams.update({'font.size': 16})

In [2]:
ifn = r"C:\data\LAPD\JAN2024_diverging_B\02_2cmXYline_M1P24_M3P27_2024-01-26_15.16.39.hdf5"

with lapd.File(ifn) as f:
    rh.show_info(f)

pr_area = {
    'M3': np.array([1,1,1,1.15]) * 0.2*0.4,
    'M1': np.array([1,1,1.14,1]) * 0.1*0.4
} # [UL, UR, LL, LR] ; unit:cm^2

02_2cmXYline_M1P24_M3P27_2024-01-26_15.16.39.hdf5 Overview
Generated by bapsflib (v0.0.0)
Generated date: 8/15/2024 11:10:59 AM


Filename:     02_2cmXYline_M1P24_M3P27_2024-01-26_15.16.39.hdf5
Abs. Path:    C:\data\LAPD\JAN2024_diverging_B\02_2cmXYline_M1P24_M3P27_2024-01-26_15.16.39.hdf5
LaPD version: 1.2
Investigator: Jia Han
Run Date:     1/26/2024 3:11:31 PM

Exp. and Run Structure:
  (set)  Plasma_Flow
  (exp)  +-- JAN2024_diverging_B
  (run)  |   +-- 02_2cmXYline_M1P24_M3P27

Run Description:
    corse XY line with 4-tip mach probe under gradient field plasma condition.
    2 probe Port 24 and 27, upper tips Isat, lower tips sweep
    
    LAPD B field:
    Black magnets at south:(PS12-13: 555A)  1 kG
    Yellow 1,2: 1kG (2600A)
    Magenta 5-7: 	1kG (910A)
    Magenta 8-10: 400G (350A)
    Yellow 3,4: 400G (1000A)
    Black magnet at north PS11: (0 A) 	0.0 kG
    
    South LaB6 source:
    H plasma (150 on MFC dial)
    He gas puff using Piezo near south anode (~1" size cu tub

In [3]:
adc, xpos, ypos, zpos, npos, nshot, int_arr, int_tarr = init_read(ifn)

SIS Crate activated:
3302 board 1
Channel 1 -- active: TRUE -- description: p27_M3_UL
Channel 2 -- active: TRUE -- description: p27_M3_UR
Channel 3 -- active: TRUE -- description: p27_M3_LL
Channel 4 -- active: TRUE -- description: p27_M3_LR
Channel 5 -- active: FALSE -- description:  
Channel 6 -- active: FALSE -- description:  
Channel 7 -- active: TRUE -- description: Vsweep_supply
Channel 8 -- active: FALSE -- description:  
3302 board 2
Channel 1 -- active: TRUE -- description: p24_M1_UL
Channel 2 -- active: TRUE -- description: p24_M1_UR
Channel 3 -- active: TRUE -- description: p24_M1_LL
Channel 4 -- active: TRUE -- description: p24_M1_LR
Channel 5 -- active: FALSE -- description:  
Channel 6 -- active: FALSE -- description:  
Channel 7 -- active: FALSE -- description:  
Channel 8 -- active: FALSE -- description:  
2 probes found.
Langmuir Marvin_3  at port  27
Only one motion list found: Xline-Y0-dx2-Nx31
Number of positions: 31
Number of shots per position: 11
This probe moves

c:\Users\hjia9\Documents\GitHub\data-analysis\.venv\Lib\site-packages\bapsflib-0.0.0-py3.11.egg\bapsflib\_hdf\utils\hdfreadcontrols.py:315: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  data = np.empty(shape, dtype=dtype)
c:\Users\hjia9\Documents\GitHub\data-analysis\.venv\Lib\site-packages\bapsflib-0.0.0-py3.11.egg\bapsflib\_hdf\utils\hdfreadcontrols.py:315: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  data = np.empty(shape, dtype=dtype)


Number of shots per position: 11
This probe moves after previous probe finish taking data


In [4]:
with lapd.File(ifn) as f:
    tarr, Vsweep_arr, IsweepL_dic, IsweepR_dic = get_IV(f, adc, npos, nshot, pr_area, cal_fac = [1, 1.414])

c:\Users\hjia9\Documents\GitHub\data-analysis\.venv\Lib\site-packages\bapsflib-0.0.0-py3.11.egg\bapsflib\_hdf\utils\hdfreadcontrols.py:315: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  data = np.empty(shape, dtype=dtype)
c:\Users\hjia9\Documents\GitHub\data-analysis\.venv\Lib\site-packages\bapsflib-0.0.0-py3.11.egg\bapsflib\_hdf\utils\hdfreaddata.py:508: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  data = np.empty(shape, dtype=dtype)
c:\Users\hjia9\Documents\GitHub\data-analysis\.venv\Lib\site-packages\bapsflib-0.0.0-py3.11.egg\bapsflib\_hdf\utils\hdfreadcontrols.py:315: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  data = np.empty(shap

In [15]:
# def smooth_IV(Vsweep, Isweep, position_range, nshot):

start_t_ls, stop_t_ls = find_IV_tndx(Vsweep_arr[0])
# Initialize an empty list to store the chunks
I_chunks = []
V_chunks = []

# Loop through the pairs of start_t_ls and stop_t_ls
for start, stop in zip(start_t_ls, stop_t_ls):

    I_chunks.append(IsweepL_dic[1][:, :, start:stop])
    V_chunks.append(Vsweep_arr[:, start:stop])

# Stack the list of chunks into a new array
Isweep_reshaped = np.stack(I_chunks, axis=2)
Vsweep_reshaped = np.stack(V_chunks, axis=1)

# Check the shape of the reshaped array
print(Isweep_reshaped.shape, Vsweep_reshaped.shape)

(31, 11, 4, 1400) (31, 4, 1400)


In [16]:
pndx = 20
shndx = 0
plt.figure()
for i in range(4):
    plt.plot(Vsweep_reshaped[pndx,i], Isweep_reshaped[pndx, shndx,i])

In [5]:
def get_dIdV_all(Vsweep, Isweep, position_range, nshot, i=-1):

    dIdV_dic = {}
    Vnew_dic = {}

    for pos_idx, pos in enumerate(position_range):
        
        start_t_ls, stop_t_ls = find_IV_tndx(Vsweep[pos])
        start = start_t_ls[i]
        stop = stop_t_ls[i]
        Vswp = Vsweep[pos, start:stop]

        dIdV_ls = []
        max_inds = []
        for j in range(nshot):
            Iswp = Isweep[pos, j, start:stop]
            ss_Iswp = savgol_filter(Iswp, window_length=40, polyorder=3)

            dIdV, max_ind = derivative(Vswp, ss_Iswp) # dIdV is gaussian smoothed by default
            dIdV_ls.append(dIdV[:max_ind])
            max_inds.append(max_ind)

        min_max_ind = min(max_inds)
        min_length = min(len(dIdV) for dIdV in dIdV_ls)
        dIdV_resized = np.zeros((nshot, min_length))

        # Resize all dIdV arrays to keep elements from a to min_max_ind
        dIdV_resized = np.array([d[max_ind - min_length:max_ind] for d, max_ind in zip(dIdV_ls, max_inds)])
        Vswp_resized = Vswp[min_max_ind - min_length:min_max_ind]
        
        # Average over all shot numbers
        dIdV_avg = np.mean(dIdV_resized, axis=0)
        
        # Store the results in dictionaries
        dIdV_dic[pos] = dIdV_avg
        Vnew_dic[pos] = Vswp_resized - np.max(Vswp_resized)
    
    return dIdV_dic, Vnew_dic

In [7]:
# Define the position range as a choice
position_range = range(12, 17)  # You can change this range as needed
bd = 2

dIdV_UL, Vnew_UL = get_dIdV_all(Vsweep_arr, -Isweep_UL_dic[bd], position_range, nshot)
dIdV_UR, Vnew_UR = get_dIdV_all(Vsweep_arr, -Isweep_UR_dic[bd], position_range, nshot)

ValueError: zero-size array to reduction operation maximum which has no identity

In [41]:
pndx = position_range[5]

plt.figure(2)
plt.title('x = ' + str(xpos[pndx]))
plt.plot(Vnew_UL[pndx], dIdV_UL[pndx], label='L'+str(bd))
plt.plot(Vnew_UR[pndx], dIdV_UR[pndx], label='R'+str(bd))
plt.legend()

In [12]:
bd = 1
Te_avg, Te_err = get_Te_all(Vsweep_arr, Isweep_UR_dic[bd], position_range, nshot)

plt.figure()
for i in range(4):
    plt.plot(xpos[position_range], Te_avg[i])
    plt.errorbar(xpos[position_range], Te_avg[i], yerr=Te_err[i], fmt='o')

In [19]:
with lapd.File(ifn) as f:
    tarr, Isat_UL_dic, Isat_UR_dic, I_ratio = get_Isat_ratio(f, adc, npos, nshot, pr_area, R=[7.8,9.7], bg_tind=60000)

c:\Users\hjia9\Documents\GitHub\data-analysis\.venv\Lib\site-packages\bapsflib-0.0.0-py3.11.egg\bapsflib\_hdf\utils\hdfreadcontrols.py:315: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  data = np.empty(shape, dtype=dtype)
c:\Users\hjia9\Documents\GitHub\data-analysis\.venv\Lib\site-packages\bapsflib-0.0.0-py3.11.egg\bapsflib\_hdf\utils\hdfreaddata.py:508: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  data = np.empty(shape, dtype=dtype)
c:\Users\hjia9\Documents\GitHub\data-analysis\.venv\Lib\site-packages\bapsflib-0.0.0-py3.11.egg\bapsflib\_hdf\utils\hdfreadcontrols.py:315: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  data = np.empty(shap

In [20]:
start_ls = [1000, 12000, 22000, 32000, 42000]
stop_ls = [50000, 13000, 23000, 33000, 43000]
pos_ind = 15

plt.figure()
plt.title('Isat ratio at x=0,y=0')
plt.xlabel('t (ms)')
plt.ylabel('I_s/I_n')

plt.plot(tarr[start_ls[0]:stop_ls[0]]*1e3, I_ratio[1][pos_ind,start_ls[0]:stop_ls[0]], label='P27')
plt.plot(tarr[start_ls[0]:stop_ls[0]]*1e3, I_ratio[2][pos_ind,start_ls[0]:stop_ls[0]], label='P24')

plt.ylim([1, 3.5])

plt.legend()
plt.grid(True)
plt.tight_layout()

for i in range(1, 5):
    plt.plot(tarr[start_ls[i]:stop_ls[i]]*1e3, I_ratio[1][pos_ind,start_ls[i]:stop_ls[i]], color='black')
    plt.plot(tarr[start_ls[i]:stop_ls[i]]*1e3, I_ratio[2][pos_ind,start_ls[i]:stop_ls[i]], color='black')

In [9]:
colors = plt.cm.rainbow(np.linspace(0, 1, 5))
ax = setup_2subplots(ylabel='I_s/I_n')

for i in range(1,5):
    I_ratio_avg = np.mean(I_ratio[1][:,start_ls[i]:stop_ls[i]], axis=-1)
    ax[0].scatter(xpos, I_ratio_avg, marker='o', color=colors[i], label='t'+str(i))

    I_ratio_avg = np.mean(I_ratio[2][:,start_ls[i]:stop_ls[i]], axis=-1)
    ax[1].scatter(xpos, I_ratio_avg, marker='^', color=colors[i], label='t'+str(i))
        

ax[1].legend(loc='lower center', ncol=4, handletextpad=0.1, columnspacing=0.1)

# ax[0].set_ylim([1, 3.5])
# ax[1].set_ylim([1, 2.1])
plt.xlim([-25, 25])
plt.tight_layout()

In [23]:
colors = plt.cm.rainbow(np.linspace(0, 1, 5))
ax = setup_2subplots(ylabel="I_sat (A/cm^2)")

for i in range(1,5):
    sig = np.mean(Isat_UL_dic[1][:,:,start_ls[i]:stop_ls[i]], axis=(1,2))
    ax[0].scatter(xpos, sig, marker='o', color=colors[i], label='t'+str(i))

    sig = np.mean(Isat_UL_dic[2][:,:,start_ls[i]:stop_ls[i]], axis=(1,2))
    ax[1].scatter(xpos, sig, marker='^', color=colors[i], label='t'+str(i))

ax[1].legend(loc='upper center', bbox_to_anchor=(0.08, 1.2), handletextpad=0.1, columnspacing=0.1)
ax[0].set_ylim([0, 2.2])
ax[1].set_ylim([0, 2.2])
plt.tight_layout()

In [21]:
with h5py.File(ifn, 'r') as f:
    message_array, status_array, timestamp_array = rh.unpack_datarun_sequence(f)

In [22]:
print(message_array[0:10])

['DAQ controller: Request=Run device background, Device=6K Compumotor'
 '6K Compumotor: Request=DAQ control, Data run key=data_run_id 237 Plasma_Flow: Timeout (sec)=60'
 'DAQ controller: Request=Run device background, Device=SIS crate'
 'SIS crate: Request=DAQ control, Data run key=data_run_id 237 Plasma_Flow: Timeout (sec)=60'
 'SIS crate: Request=Load configuration, Configuration name=siscf1-2b-9ch-noavg-longT'
 'SIS crate: Request=Create data file, Experiment set=Plasma_Flow, Experiment=JAN2024_diverging_B, Data run=02_2cmXYline_'
 '6K Compumotor: Request=Load connections'
 '6K Compumotor: Request=Select receptacle, Receptacle=1'
 '6K Compumotor: Request=Load xy motion list, Motion list=Xline-Y0-dx2-Nx31'
 '6K Compumotor: Request=Select receptacle, Receptacle=1']
